In [12]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [35]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    # Create a usable list of points from the contours
    contour_points = []

    for contour in contours[0]:
        contour_points.append(contour[0])

    contour_points = np.array(contour_points)

    # Shrink the list of contours 
    shrinked_list = []

    for i in range(0, len(contour_points)):
        if i % 10 == 0: shrinked_list.append(contour_points[i])

    return np.array(shrinked_list)

In [26]:
# Create a named colour
img = cv2.imread('masks/4_1.png')
contours = find_contours(img)

red = [0,0,255]

cv2.polylines(img, [contours], True, red, 3)

# Show
cv2.imshow('image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
img = cv2.imread('masks/4_1.png')
contours = find_contours(img)

print(contours.transpose())

[]


In [42]:
# Save the points to a json file
images = []
image_names = []

for file in glob.glob("./masks/*.png"):
    images.append(cv2.imread(file))
    image_names.append(file.split('/')[2])

data = {}

for filename, image in zip(image_names, images):
    contours = find_contours(image)
    if len(contours) != 0:
        region = {}
        region["name"] = "polyline"
        region["all_points_x"] = contours.transpose()[0].tolist()
        region["all_points_y"] = contours.transpose()[1].tolist()

        image_information = {}
        image_information["filename"] = filename
        image_information["class"] = filename.split('_')[1].split('.')[0]
        image_information["region"] = region

        data["image_" + filename] = image_information

with open('annotations.json', 'w') as json_file:
    json.dump(data, json_file)